In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, Lasso, Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.gaussian_process.kernels import Matern, RBF
from sklearn.tree import DecisionTreeRegressor

from xgboost import XGBRegressor

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from doc.mse_estimator import ErrorComparer
from doc.data_generation import gen_rbf_X, gen_matern_X, create_clus_split, gen_cov_mat
from doc.plotting_utils import gen_model_barplots
from spe.estimators import kfoldcv, kmeanscv, better_test_est_split, cp_general_train_test, by_spatial
from spe.relaxed_lasso import RelaxedLasso

In [2]:
np.random.seed(1)

In [7]:
## number of realizations to run
niter = 100

## data generation parameters
gsize=10
n=10**2
p=200
s=200
delta = 0.75
snr = 2.
tr_frac = .25

noise_kernel = 'matern'
noise_length_scale = 1.
noise_nu = .5

X_kernel = 'matern'
X_length_scale = 5.
X_nu = 2.5

## ErrorComparer parameters
alpha = .05
nboot = 100
k = 5

models = [DecisionTreeRegressor(max_depth=5, max_features='sqrt')]

ests = [
    better_test_est_split,
    better_test_est_split,
    cp_general_train_test,
    by_spatial,
    by_spatial,
    by_spatial,
]
est_kwargs = [
    {'alpha': None,
    'full_refit': False},
    {'alpha': .05,
    'full_refit': False},
    {'alpha': alpha, 
    'use_trace_corr': False, 
    'nboot': nboot},
    {'alpha': .05, 
    'nboot': nboot},
    {'alpha': 1., 
    'nboot': nboot},
    {'alpha': 5., 
    'nboot': nboot},
]

## plot parameters
est_names = ["GenCp .05", "BY .05", "BY 1.", "BY 5."]
model_names = ["Decision Tree"]

In [8]:
err_cmp = ErrorComparer()

In [9]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, gsize, nx)
ys = np.linspace(0, gsize, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

In [10]:
if noise_kernel == 'rbf':
    Sigma_t = gen_cov_mat(c_x, c_y, RBF(length_scale=noise_length_scale))
elif noise_kernel == 'matern':
    Sigma_t = gen_cov_mat(c_x, c_y, Matern(length_scale=noise_length_scale, nu=noise_nu))
else:
    Sigma_t = np.eye(n)
    
Cov_st = delta*Sigma_t
Sigma_t = delta*Sigma_t + (1-delta)*np.eye(n)

if noise_kernel == 'rbf' or noise_kernel == 'matern':
    Chol_t = np.linalg.cholesky(Sigma_t)
else:
    Chol_t = np.eye(n)

In [11]:
if X_kernel == 'rbf':
    Sigma_X = gen_cov_mat(c_x, c_y, RBF(length_scale=X_length_scale))
elif X_kernel == 'matern':
    Sigma_X = gen_cov_mat(c_x, c_y, Matern(length_scale=X_length_scale, nu=X_nu))
else:
    Sigma_X = np.eye(n)

if X_kernel == 'rbf' or X_kernel == 'matern':
    Chol_X = np.linalg.cholesky(Sigma_X)
else:
    Chol_X = np.eye(n)

X = Chol_X @ np.random.randn(n,p)

if X_kernel == 'rbf':
    X_spikes = gen_rbf_X(c_x, c_y, p)
elif X_kernel == 'matern':
    X_spikes = gen_matern_X(c_x, c_y, p, length_scale=X_length_scale, nu=X_nu)
else:
    X_spikes = np.random.randn(n,p)

X_iso = np.random.randn(n,p)

In [12]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 30, nx)
ys = np.linspace(0, 30, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

if X_kernel == 'rbf':
    Sigma_X_less = gen_cov_mat(c_x, c_y, RBF(length_scale=X_length_scale))
elif X_kernel == 'matern':
    Sigma_X_less = gen_cov_mat(c_x, c_y, Matern(length_scale=X_length_scale, nu=X_nu))
else:
    Sigma_X_less = np.eye(n)

if X_kernel == 'rbf' or X_kernel == 'matern':
    Chol_X_less = np.linalg.cholesky(Sigma_X_less)
else:
    Chol_X_less = np.eye(n)

X_less = Chol_X_less @ np.random.randn(n,p)

In [13]:
beta = np.zeros(p)
idx = np.random.choice(p,size=s,replace=False)
beta[idx] = np.random.uniform(-1,1,size=s)

In [14]:
tr_idx = np.ones(n, dtype=bool)

# Simulate $\begin{pmatrix} Y \\ Y^* \end{pmatrix} \sim \mathcal{N}\left(\begin{pmatrix} \mu \\ \mu \end{pmatrix}, \begin{pmatrix}\Sigma_Y & \Sigma_{Y, Y^*} \\ \Sigma_{Y^*, Y} & \Sigma_{Y}  \end{pmatrix}\right)$

## $X_{\cdot,i}$ independently generated by uniform spikes at locations, then interpolate based on cov matrix

In [15]:
spike_model_errs = []

for model in models:
    errs = err_cmp.compare(
        model,
        ests,
        est_kwargs,
        niter=niter,
        n=n,
        p=p,
        s=s,
        snr=snr, 
        X=X_spikes,
        beta=beta,
        coord=coord,
        Chol_y=Chol_t,
        Chol_ystar=None,
        Cov_y_ystar=Cov_st,
        tr_idx=tr_idx,
        fair=False,
        est_sigma=False,
    )
    spike_model_errs.append(errs)

100%|██████████| 100/100 [00:22<00:00,  4.47it/s]


In [16]:
spike_fig = gen_model_barplots(
    spike_model_errs, 
    model_names, 
    est_names, 
    title="Spike", 
    has_elev_err=True,
    err_bars=True,
)
spike_fig.show()

## $X_{i,\cdot} \sim \mathcal{N}(0, I\sigma^2)$

In [17]:
iid_model_errs = []

for model in models:
    errs = err_cmp.compare(
        model,
        ests,
        est_kwargs,
        niter=niter,
        n=n,
        p=p,
        s=s,
        snr=snr, 
        X=X_iso,
        beta=beta,
        coord=coord,
        Chol_y=Chol_t,
        Chol_ystar=None,
        Cov_y_ystar=Cov_st,
        tr_idx=tr_idx,
        fair=False,
        est_sigma=False,
    )
    iid_model_errs.append(errs)

100%|██████████| 100/100 [00:23<00:00,  4.23it/s]


In [18]:
iid_fig = gen_model_barplots(
    iid_model_errs, 
    model_names, 
    est_names, 
    title="IID ",
    has_elev_err=True,
    err_bars=True,
)
iid_fig.show()

## $X_{\cdot,i} \sim \mathcal{N}(0, \Sigma), i=1,\dots,p$

In [21]:
model_errs = []

for model in models:
    errs = err_cmp.compare(
        model,
        ests,
        est_kwargs,
        niter=niter,
        n=n,
        p=p,
        s=s,
        snr=snr, 
        X=X,
        beta=beta,
        coord=coord,
        Chol_y=Chol_t,
        Chol_ystar=None,
        Cov_y_ystar=Cov_st,
        tr_idx=tr_idx,
        fair=False,
        est_sigma=False,
    )
    model_errs.append(errs)

100%|██████████| 100/100 [00:22<00:00,  4.52it/s]


In [22]:
fig = gen_model_barplots(
    model_errs, 
    model_names, 
    est_names, 
    title="Cov",
    has_elev_err=True,
    err_bars=True,
)
fig.show()

## $X_{\cdot,i} \sim \mathcal{N}(0, \Sigma'), i=1,\dots,p$ but $\Sigma'$ less covariance than $\Sigma$ 

In [23]:
less_model_errs = []

for model in models:
    errs = err_cmp.compare(
        model,
        ests,
        est_kwargs,
        niter=niter,
        n=n,
        p=p,
        s=s,
        snr=snr, 
        X=X_less,
        beta=beta,
        coord=coord,
        Chol_y=Chol_t,
        Chol_ystar=None,
        Cov_y_ystar=Cov_st,
        tr_idx=tr_idx,
        fair=False,
        est_sigma=False,
    )
    less_model_errs.append(errs)

100%|██████████| 100/100 [00:22<00:00,  4.50it/s]


In [24]:
less_fig = gen_model_barplots(
    less_model_errs,
    model_names,
    est_names,
    title="Less Cov ",
    has_elev_err=True,
    err_bars=True,
)
less_fig.show()

In [25]:
less_fig = gen_model_barplots(
    model_errs + less_model_errs,
    ["Cov", "Less"],
    est_names,
    title="BY Comparisons: ",
    has_elev_err=True,
    err_bars=True,
)
less_fig.show()

In [29]:
less_fig = gen_model_barplots(
    spike_model_errs + iid_model_errs,
    ["Corr X", "IID X"],
    est_names,
    title="BY Comparisons: Depth 5 Decision Tree, SSN",
    has_elev_err=True,
    err_bars=True,
)
less_fig.show()